# Orchestral AI Demo 2

This notebook shows how to use Orchestral AI to make an AI agent that can interact with the SBND electronic logbook and create files on the local computer.

In [1]:
from pathlib import Path

from orchestral import Agent
from orchestral.llm import *
from orchestral import define_tool
from tool_logger import ToolCallLogger
from orchestral.tools import (
    RunCommandTool, WriteFileTool, ReadFileTool, EditFileTool,
    FileSearchTool,  FindFilesTool, RunPythonTool, WebSearchTool,
    TodoWrite, TodoRead
)



In [2]:
with open('pwd.txt', 'r') as f:
    ecl_pwd = f.read().strip()  


In [ ]:
@define_tool()
def ecl_search(category='Shift', limit=3, after=None):
    """
    Searches the SBND electronic logbook (ECL) for all the first `limit` entries of a certain `category`.

    Args:
        category: The ECL category
        limit: Limit to the number of entries to retrieve
        after (str): Searches for entries after a certain date. The date has to be in the following formats:
            <n>days (ex: "1days" for the last 24h entries)
            <n>hours (ex: "1hours" for the last hour entries)
            <n>minutes (ex: "1minutes" for the last minute entries)
            yyyy-mm-dd+hh:mm:ss (ex: "2012-04-01+12:00:00"

    Returns:
        List of entries

    NOTE: If a category is not provide, use "Shift" as default.
    """
    from ecl_api import ECL, ECLEntry
    import xml.etree.ElementTree as ET
    
    password = ecl_pwd
    url = "https://dbweb1.fnal.gov:8443/ECL/sbnd/E"

    ecl = ECL(url=url, user='sbndprm', password=password)

    # return [ecl.get_entry(entry_id=7252)]
    text = ecl.search(category=category, limit=limit, after=after)

    xml = ET.fromstring(text)
    entries = xml.findall('./entry')
    result = []
    for entry in entries:
        result.append((entry.attrib, entry.tag))
    return result

In [4]:
hooks = [ToolCallLogger(
        verbose=True,       # Show full arguments (False = just tool names)
        show_results=False  # Show tool outputs (can be very verbose)
    )]

In [5]:
base_directory = "./workdir/"
Path(base_directory).mkdir(exist_ok=True)

In [6]:
tools = [
    # Core file/command tools
    RunCommandTool(base_directory=base_directory),
    WriteFileTool(base_directory=base_directory),
    ReadFileTool(base_directory=base_directory, show_line_numbers=True),
    EditFileTool(base_directory=base_directory),
    FindFilesTool(base_directory=base_directory),
    FileSearchTool(base_directory=base_directory),
    RunPythonTool(base_directory=base_directory, timeout=1000),
    # WebSearchTool(),
    TodoRead(),
    TodoWrite(base_directory=base_directory),
    # Custom ECL search tool
    ecl_search,
]

In [7]:
agent = Agent(
    llm=Ollama(),
    tools=tools,
    tool_hooks=hooks,
    system_prompt="You are a helpful physics assistant with knowledge on how to retrive entries from the logbook."
)


In [8]:
response = agent.run('What is the ecl_search function?')
print(response.text)

**`ecl_search` – a built‑in helper to pull entries from the SBND electronic logbook (ECL).**

> **Purpose**  
> Quickly query the SBND ECL for the most recent log entries that belong to a specific category (e.g., *“experiment”*, *“maintenance”*, *“analysis”*). It returns a short list of the first `limit` entries that match, optionally filtered to those created after a given timestamp.

| Argument | Type | Description |
|----------|------|-------------|
| `category` | string | The name of the logbook category you want to search (e.g., `"experiment"`, `"shifts"`, `"maintenance"`). |
| `limit` | integer | How many entries to return. Only the first `limit` entries (ordered most‑recent‑first) will be delivered. |
| `after` | string | Optional. Restricts the search to entries created **after** a specified point in time. Accepted formats:<br>• `"<n>days"` – e.g., `"1days"` for the last 24 h.<br>• `"<n>hours"` – e.g., `"12hours"`.<br>• `"<n>minutes"` – e.g., `"30minutes"`.<br>• Absolute timest

In [9]:
response = agent.run('Can you fetch the last 10 entries and give me the result?')
print(response.text)

Sure! To pull the last 10 entries, I’ll need to know which ECL **category** you’re interested in (e.g., *experiment*, *maintenance*, *shifts*, etc.). Once you let me know the category, I can run the query and return the most recent 10 entries.


In [10]:
response = agent.run('Please use Shift as category')
print(response.text)


>>> TOOL CALL: ecl_search
    Arguments: {
  "category": "Shift",
  "limit": 10,
  "after": "0days"
}
<<< ecl_search completed

>>> TOOL CALL: ecl_search
    Arguments: {
  "after": "0days",
  "category": "shift",
  "limit": 10
}
<<< ecl_search completed

>>> TOOL CALL: ecl_search
    Arguments: {
  "after": "0days",
  "category": "shifts",
  "limit": 10
}
<<< ecl_search completed

>>> TOOL CALL: ecl_search
    Arguments: {
  "after": "-1days",
  "category": "shift",
  "limit": 10
}
<<< ecl_search completed

>>> TOOL CALL: ecl_search
    Arguments: {
  "after": "0days",
  "category": "Shift",
  "limit": 10
}
<<< ecl_search completed

>>> TOOL CALL: ecl_search
    Arguments: {
  "after": "0hours",
  "category": "shift",
  "limit": 10
}
<<< ecl_search completed

>>> TOOL CALL: ecl_search
    Arguments: {
  "after": "30days",
  "category": "Shift",
  "limit": 10
}
<<< ecl_search completed
Here are the **10 most recent Shift entries** (sorted from newest to oldest):

| # | Entry ID | Auth

In [11]:
response = agent.run('Can you create a text file with each line containing a summary of the logbook entries you just fetched?')
print(response.text)


>>> TOOL CALL: writefile
    Arguments: {
  "path": "shift_summaries.txt",
  "data": "ID 32658 by mdeltutt at 01/28/2026 10:34:12: Beam Checklist - v2, 0 images, 0 files.\nID 32657 by mdeltutt at 01/28/2026 09:36:53: Beam Checklist - v2, 0 images, 0 files.\nID 32656 by mdeltutt at 01/28/2026 08:57:44: Shifter Checklist - Run2 - v6.2, 2 images, 0 files.\nID 32655 by mdeltutt at 01/28/2026 08:17:25: Beam Checklist - v2, 0 images, 0 files.\nID 32654 by mdeltutt at 01/28/2026 07:59:27: Start of Shift Checklist - v4, 0 images, 0 files.\nID
  ... (truncated)
<<< writefile completed
✅ **`shift_summaries.txt`** has been created in your working directory.  
Each line contains a brief summary of one of the last 10 Shift entries, formatted like this:

```
ID 32658 by mdeltutt at 01/28/2026 10:34:12: Beam Checklist - v2, 0 images, 0 files.
```

You can open the file to see all 10 summaries. Let me know if you’d like any additional formatting or extra details!
